# Model Experiments Phase - TSA 1001

#### About Notebook - Experiments_M4

1. experiment_m5: Standardized & tracked to Mlflow

#### Import Necessary Libraries

In [48]:
# Basic Libraies
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
import warnings as wr
wr.filterwarnings('ignore')

In [49]:
# Earlier was v4, lets try with v6 (v6 reduce column number, best selected with scaled)
data = pd.read_parquet("D:/Study/PROJECT-REALTIME/PT-1001/model_output_data/output/sales_bya_v6.parquet") # I guest data not be requried, need confirmation
df = data.copy()

In [50]:
df.head()

,date_month_start,net_price_lag_7,net_price_lag_28,net_price_lag_1,net_price,date
0,0,-1.076879,5.909094,0.057816,5042942.0,2019-08-08
1,0,1.824821,2.914794,0.216489,2932270.5,2019-08-10
2,0,0.207312,0.364461,-0.349414,1809937.5,2019-08-12
3,0,-1.122481,1.342665,-0.650329,3067519.0,2019-08-13
4,0,-0.434004,1.281428,-0.313152,5123492.0,2019-08-14


In [51]:
df.tail()

,date_month_start,net_price_lag_7,net_price_lag_28,net_price_lag_1,net_price,date
1116,0,1.011448,0.846822,1.806918,4.866168e+04,2023-06-25
1117,0,-0.110007,0.117538,-1.122553,2.878786e+06,2023-06-26
1118,0,0.273030,2.004125,-0.363754,4.076503e+06,2023-06-27
1119,0,0.785505,-1.128543,-0.042628,5.416796e+06,2023-06-28
1120,0,0.914636,-0.203244,0.316725,2.074890e+07,2023-06-29


In [52]:
df.shape

(1121, 6)

# Seperation X, y (features, Target) 

1. first 3 year data for Train Set
2. last 1 year data for Test set

####  a. Train Set ( 3 Fiscal years data) 

In [53]:
# Define the start and end dates of the range
start_date = '2019-07-01'
end_date = '2022-06-30'

In [54]:
# Create a mask to filter the rows that fall within the range
mask = (df['date'] >= start_date) & (df['date'] <=end_date)

In [55]:
# Apply the mask to get the filtered dataframe
train = df.loc[mask]
train.head()

,date_month_start,net_price_lag_7,net_price_lag_28,net_price_lag_1,net_price,date
0,0,-1.076879,5.909094,0.057816,5042942.0,2019-08-08
1,0,1.824821,2.914794,0.216489,2932270.5,2019-08-10
2,0,0.207312,0.364461,-0.349414,1809937.5,2019-08-12
3,0,-1.122481,1.342665,-0.650329,3067519.0,2019-08-13
4,0,-0.434004,1.281428,-0.313152,5123492.0,2019-08-14


In [56]:
train.shape

(835, 6)

####  b. Test Set ( 3 Fiscal years data)

In [57]:
# Define the start and end dates of the range
test_start = '2022-07-01'
test_end = '2023-06-30'

In [58]:
# Create a mask to filter the rows that fall within the range
mask2 = (df['date'] >= test_start) & (df['date'] <=test_end)

In [59]:
# Apply the mask to get the filtered dataframe
test = df.loc[mask2]
test.head()

,date_month_start,net_price_lag_7,net_price_lag_28,net_price_lag_1,net_price,date
835,0,0.393723,-0.320125,0.223613,1.661066e+07,2022-07-02
836,0,-1.119361,-0.030373,3.317969,5.036286e+06,2022-07-03
837,0,-0.406804,2.646675,0.214704,5.494030e+06,2022-07-04
838,0,-0.069035,0.034942,0.337432,1.113029e+05,2022-07-05
839,0,0.229674,0.197900,-1.105758,3.572892e+06,2022-07-06


In [60]:
test.shape

(286, 6)

#### Training & Testing Set

In [61]:
# Training Set
X_train = train.drop(columns=['date','net_price'])
y_train = train['net_price']

print(X_train.shape)
print(y_train.shape)

(835, 4)
(835,)


In [62]:
# Test Set
X_test = test.drop(columns=['date','net_price'])
y_test = test['net_price']

print(X_test.shape)
print(y_test.shape)

(286, 4)
(286,)


In [63]:
# Check the whether the lenge of data is same for both train & test set
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(835, 4)
(835,)
(286, 4)
(286,)


## 2.0 Define Models

In [64]:
# Define Models
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

### 3.0 Evaluation Metrics

In [65]:
# Evaluation metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

<center> <h1> 4.0 Experiments - Mlflow </h1></center>

In [66]:
#!pip install mlflow 

In [67]:
import mlflow

In [69]:
mlflow.set_tracking_uri("http://localhost:5000")

In [70]:
mlflow.create_experiment('Forecating_TSA')

'498311194986016427'

##  Modeling

In [72]:
# Start MLflow run session
mlflow.set_experiment('Forecating_TSA')
mlflow.set_tracking_uri("http://localhost:5000")
with mlflow.start_run():
    
    ## Model architecture- Model_1: Ridge
    model_1= Ridge(
    alpha=1.0, fit_intercept=True, copy_X=True, max_iter=None,
    tol=0.0001, solver='auto', positive=False, random_state=42,
    )
    
    ## Fit the model
    model_1.fit(X_train, y_train)
    
    ## Log param
    mlflow.log_params(dict(alpha=1.0, fit_intercept=True, copy_X=True, max_iter=None,
    tol=0.0001, solver='auto', positive=False, random_state=42))
    
    ## log metrics
    mlflow.log_metrics(
        {
            'r2': r2_score(model_1.predict(X_test), y_test),
            'mape': mean_absolute_percentage_error(model_1.predict(X_test), y_test),
            'mse': mean_squared_error(model_1.predict(X_test), y_test),
            'mae': mean_absolute_error(model_1.predict(X_test), y_test)
            
        }
    )
    
    ## log model
mlflow.set_experiment('Forecating_TSA')
with mlflow.start_run():
     
    # Model_2: KNeighborsRegressor
    model_2 = KNeighborsRegressor(n_neighbors=7, weights='distance', metric='euclidean')
    
    ## fit/ train the model
    model_2.fit(X_train, y_train)
    
    ## log params
    mlflow.log_params(dict(n_neighbors=7, weights='distance', metric='euclidean'))
    
    ## log metrics
    mlflow.log_metrics(
        {
            'r2': r2_score(model_2.predict(X_test), y_test), # it will cal r2_score on test data, X:prediction, y: actual
            'mape':mean_absolute_percentage_error(model_2.predict(X_test), y_test),
            'mse':mean_squared_error(model_2.predict(X_test), y_test),
            'mae':mean_absolute_error(model_2.predict(X_test), y_test)
        }
    
    )
    ## log the model
mlflow.set_experiment('Forecating_TSA')    
with mlflow.start_run():  
    # Model-3: Decision Tree
    model_3 = DecisionTreeRegressor(
    criterion='squared_error', splitter='best', 
    max_depth=3, min_samples_split=2, 
    min_samples_leaf=2, min_weight_fraction_leaf=0.0, 
    max_features=None, random_state=42, max_leaf_nodes=None, 
    min_impurity_decrease=0.0, ccp_alpha=0.0
    )
    
    ## fit/train the model
    model_3.fit(X_train, y_train)
    
    ## log params
    mlflow.log_params(
        dict(criterion='squared_error', 
             splitter='best', max_depth=3,
             min_samples_split=2,
             min_samples_leaf=2, 
             min_weight_fraction_leaf=0.0,
             max_features=None, random_state=42,
             max_leaf_nodes=None,
             min_impurity_decrease=0.0, ccp_alpha=0.0)
    )
    
    ## log metrics
    mlflow.log_metrics(
        {
            'r2':r2_score(model_3.predict(X_test), y_test),
            'mape':mean_absolute_percentage_error(model_3.predict(X_test), y_test),
            'mse':mean_squared_error(model_3.predict(X_test), y_test),
            'mae':mean_absolute_error(model_3.predict(X_test), y_test)
        }
    
    )
    
    ## log model
mlflow.set_experiment('Forecating_TSA')    
with mlflow.start_run(): 
    
    # Model-4: Random Forest
    model_4 = RandomForestRegressor(
        n_estimators=30,
        criterion='squared_error',
        max_depth=5, # max dept= 'none' limit it by 5 for optimization
        min_samples_split=2,
        min_samples_leaf=1,
        min_weight_fraction_leaf=0.0,
        max_features=1.0,
        max_leaf_nodes=None,
        min_impurity_decrease=0.0,
        bootstrap=True,
        oob_score=False,
        n_jobs=None,
        random_state=33,
        verbose=0,
        warm_start=False,
        ccp_alpha=0.0,
        
    )
    
    ## Fit/ Train the model
    model_4.fit(X_train, y_train)
    
    ## log Params
    mlflow.log_params(
        dict(n_estimators=30,
        criterion='squared_error',
        max_depth=5, 
        min_samples_split=2,
        min_samples_leaf=1,
        min_weight_fraction_leaf=0.0,
        max_features=1.0,
        max_leaf_nodes=None,
        min_impurity_decrease=0.0,
        bootstrap=True,
        oob_score=False,
        n_jobs=None,
        random_state=33,
        verbose=0,
        warm_start=False,
        ccp_alpha=0.0)
        )
                           
    
    ## log Metrics
    mlflow.log_metrics(
        {
            'r2':r2_score(model_4.predict(X_test), y_test),
            'mape':mean_absolute_percentage_error(model_4.predict(X_test), y_test),
            'mse':mean_squared_error(model_4.predict(X_test), y_test),
            'mae':mean_absolute_error(model_4.predict(X_test), y_test)
        }
    )
    
    ## log Model
    
mlflow.set_experiment('Forecating_TSA')
with mlflow.start_run():     
    # Model-5: XGBoost
    model_5 = xgb.XGBRegressor(
        objective='reg:squarederror',
        max_depth=10,
        n_estimators=10,
        seed = 123
    )
    
    ## fit the model
    model_5.fit(X_train, y_train)
    
    ## log Params
    mlflow.log_params(
        dict(objective='reg:squarederror',
        max_depth=10,
        n_estimators=10,
        seed = 123)
    )
    
    ## log Metrics
    mlflow.log_metrics(
        {
            'r2':r2_score(model_5.predict(X_test), y_test),
            'mape':mean_absolute_percentage_error(model_5.predict(X_test), y_test),
            'mse':mean_squared_error(model_5.predict(X_test), y_test),
            'mae':mean_absolute_error(model_5.predict(X_test), y_test)
        }
    )
    
    ## log model    
  

#### Ref

1. https://tslearn.readthedocs.io/en/latest/gen_modules/neighbors/tslearn.neighbors.KNeighborsTimeSeriesRegressor.h